In [1]:
!pip install rarfile evaluate wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
import transformers
import requests
import pickle
import io
from evaluate import load
import pandas as pd
import requests
import numpy as np
import io
import rarfile
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from google.colab import drive
import os
import gc

base_google_drive_path=''
url = "https://raw.githubusercontent.com/MichalBrodackiPJA/ZastosowaniaUczeniaMaszynowego/main"

<a href="https://colab.research.google.com/github/MichalBrodackiPJA/ZastosowaniaUczeniaMaszynowego/blob/main/ZUM_Projekt_Model_referencyjny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
political_tweets_for_bert_df = pd.read_csv(f'{url}/political_tweets_for_bert.csv')

political_tweets_for_bert_df['text'] = political_tweets_for_bert_df['text'].astype(str)
political_tweets_for_bert_df.head(5)

,text,labels
0,"Happy th birthday @USNavy! strength, dedicatio...",1
1,greatest generation's investment infrastructur...,1
2,"Thanks @SenTedCruz @SenatorWarnock, Infrastruc...",0
3,/ get lasting change cant lock convicted crime...,1
4,Today celebrating years Hoosier state. Happy b...,0


In [7]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 1000

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
# tokenizer.save_pretrained("tokenizer_bert")
# tokenizer = transformers.AutoTokenizer.from_pretrained("tokenizer_bert")

In [9]:
import tensorflow as tf

def tokenize_data(examples):
    tokenized = tokenizer(examples['text'], return_tensors='tf', padding="max_length", truncation=True, max_length=45)
    # Squeeze the attention_mask to 1 dimension
    #tokenized['attention_mask'] = tf.squeeze(tokenized['attention_mask'], axis=0)
    return tokenized

tweets_tokenized_dataset = Dataset.from_pandas(political_tweets_for_bert_df)
tweets_tokenized_dataset = tweets_tokenized_dataset.map(tokenize_data, batched = True)


Map:   0%|          | 0/190491 [00:00<?, ? examples/s]

In [10]:
train_tweets_tokenized_dataset, val_tweets_tokenized_dataset = tweets_tokenized_dataset.train_test_split(test_size=0.06, seed=42).values()

#wyłączam jedną próbkę w celu demonstracyjnego przetestowania
test_tweet_tokenized_sample = val_tweets_tokenized_dataset.select([0])
val_tweets_tokenized_dataset = val_tweets_tokenized_dataset.select(range(1, len(val_tweets_tokenized_dataset)))

In [11]:

with open(f'test_tweet_tokenized_sample.pkl', 'wb') as f:
    pickle.dump(test_tweet_tokenized_sample, f)
with open(f'val_tweets_tokenized_dataset.pkl', 'wb') as f:
    pickle.dump(val_tweets_tokenized_dataset, f)
with open(f'train_tweets_tokenized_dataset.pkl', 'wb') as f:
    pickle.dump(train_tweets_tokenized_dataset, f)

In [ ]:
# url = "https://raw.githubusercontent.com/MichalBrodackiPJA/ZastosowaniaUczeniaMaszynowego/main"

# def load_from_github(filename):

#     full_url = f"{url}/{filename}"
#     response = requests.get(full_url)
#     response.raise_for_status()
#     data = pickle.load(io.BytesIO(response.content))
#     return data

# test_tweet_tokenized_sample = load_from_github('test_tweet_tokenized_sample.pkl')
# val_tweets_tokenized_dataset = load_from_github('val_tweets_tokenized_dataset.pkl')
# train_tweets_tokenized_dataset = load_from_github('train_tweets_tokenized_dataset.pkl')

In [12]:
del political_tweets_for_bert_df
del tweets_tokenized_dataset
gc.collect()

22

In [13]:
print(test_tweet_tokenized_sample.shape)
print(val_tweets_tokenized_dataset.shape)
print(train_tweets_tokenized_dataset.shape)
print(train_tweets_tokenized_dataset[0])
#print(train_tweets_tokenized_dataset['attention_mask'])
print(train_tweets_tokenized_dataset)

(1, 4)
(11429, 4)
(179061, 4)
{'text': 'CJS Appropriations Subcommittee hearing, questioned U.S. Trade Representative Tai impact U.S. seafood competitiveness hold China accountable failing commitments. ', 'labels': 0, 'input_ids': [101, 1039, 22578, 22713, 18052, 4994, 1010, 8781, 1057, 1012, 1055, 1012, 3119, 4387, 13843, 4254, 1057, 1012, 1055, 1012, 23621, 6975, 2791, 2907, 2859, 26771, 7989, 17786, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 179061
})


In [14]:
metric = load("glue", "sst2")

# Funkcja metryki
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
base_google_drive_path = '/content/drive/MyDrive/'

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

training_args = TrainingArguments(
    output_dir=base_google_drive_path,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tweets_tokenized_dataset,
    eval_dataset=val_tweets_tokenized_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-44a895ce8bf9>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

trainer.save_model('tweets_authors_party_affiliation_classifier')

Step,Training Loss,Validation Loss,Accuracy
200,No log,0.328543,0.856855


KeyboardInterrupt: 

In [21]:
trainer.save_model('tweets_authors_party_affiliation_classifier2')